In [1]:
from z3 import *
from tqdm import tqdm
import itertools

## Sorts

In [2]:
# create uninterpreted sort Node and Epoch
Node = DeclareSort('Node')
Epoch = DeclareSort('Epoch')

## State and Model

In [3]:
class DistLockState():
    def __init__(self, name):
        self.name = name

        # relations
        self.ep = Function(f'{name}.ep', Node, Epoch)
        self.held = Function(f'{name}.held', Node, BoolSort())
        self.transfer = Function(f'{name}.transfer', Epoch, Node, BoolSort())
        self.locked = Function(f'{name}.locked', Epoch, Node, BoolSort())

DistLockState('test_pre') # only for testing

In [4]:
class DistLockModel():
    def __init__(self):
        # constants
        self.le = Function(f'le', Epoch, Epoch, BoolSort())
        self.zero = Const(f'zero', Epoch)
        self.one = Const(f'one', Epoch)
        self.first = Const(f'first', Node)

        self.states = {}
    
    def get_state(self, name):
        if name not in self.states:
            self.states[name] = DistLockState(name)
        return self.states[name]
    
    def get_axioms(self):
        e1, e2, e3 = Consts('e1 e2 e3', Epoch)
        
        Axioms = ForAll([e1, e2, e3],
            And(
                # reflexivity
                self.le(e1, e1),
                # transitivity
                Implies(And(self.le(e1, e2), self.le(e2, e3)), self.le(e1, e3)),
                # antisymmetry
                Implies(And(self.le(e1, e2), self.le(e2, e1)), e1 == e2),
                # totality
                Or(self.le(e1, e2), self.le(e2, e1)),

                # zero
                self.le(self.zero, e1),
                self.one != self.zero,
            ),
        )

        return Axioms
    
    def get_init_state_cond(self):
        S = self.get_state('init')

        n = Const('n', Node)
        e = Const('e', Epoch)

        cond = ForAll([n, e],
            And(
                S.held(n) == (n == self.first),
                Implies(n != self.first, S.ep(n) == self.zero),
                S.ep(self.first) == self.one,
                S.transfer(e, n) == False,
                S.locked(e, n) == False,
            )
        )

        return cond
    
    def get_interp(self, model: ModelRef):
        # create a dict of all the functions
        interp = {}
        for f in model.decls():
            interp[f.name()] = model.get_interp(f)
        
        return interp

M = DistLockModel()
M.get_axioms() # only for testing

ForAll([e1, e2, e3],
       And(le(e1, e1),
           Implies(And(le(e1, e2), le(e2, e3)), le(e1, e3)),
           Implies(And(le(e1, e2), le(e2, e1)), e1 == e2),
           Or(le(e1, e2), le(e2, e1)),
           le(zero, e1),
           one != zero))

## Invariants

### Swiss Invariants

In [5]:
def inv_fn_0(M, S, model = None, nodes = None, epochs = None):
    def body(N, E):
        return Implies(S.locked(E,N),S.transfer(E,N))
    
    if nodes is not None:
        return all([model.eval(body(n,e)) for n in nodes for e in epochs])
    else:
        N = Const('N', Node)
        E = Const('E', Epoch)
        inv = ForAll([N, E], body(N, E))
        return inv

def inv_fn_1(M, S, model=None, nodes=None, epochs=None):
    """
    conjecture (held(N) & held(M) -> N=M)
        & ((transfer(E1, N1) & ~le(E1, ep(N1)) & transfer(E2, N2) & ~le(E2, ep(N2))) -> (E1=E2 & N1=N2))
        & (( transfer(E1, N1) & ~le(E1, ep(N1))) -> ~held(N2))
    """
    N, eM, N1, N2 = Consts('N M N1 N2', Node)
    E1, E2 = Consts('E1 E2', Epoch)

    def body(N, eM, N1, N2, E1, E2):
        return And(
            Implies(
                And(S.held(N), S.held(eM)),
                N == eM
            ),
            Implies(
                And(
                    S.transfer(E1, N1),
                    Not(M.le(E1, S.ep(N1))),
                    S.transfer(E2, N2),
                    Not(M.le(E2, S.ep(N2)))
                ),
                And(
                    E1 == E2,
                    N1 == N2,
                )
            ),
            Implies(
                And(
                    S.transfer(E1, N1),
                    Not(M.le(E1, S.ep(N1)))
                ),
                Not(S.held(N2))
            )
        )
    
    if nodes is not None:
        return all([model.eval(body(n,eM,n1,n2,e1,e2)) for n, eM, n1, n2, e1, e2 in itertools.product(nodes, nodes, nodes, nodes, epochs, epochs)])

    inv = ForAll([N, eM, N1, N2, E1, E2], body(N, eM, N1, N2, E1, E2))
    return inv

def inv_fn_2(M, S, model=None, nodes=None, epochs=None):
    """
    conjecture (( transfer(E, N) & ~le(E, ep(N))) & transfer(E1, N1) & (N~=N1 | E~=E1) -> ~le(E,E1))
            & (held(N) & transfer(E1,N1) -> le(E1,ep(N)))
    """

    N, N1 = Consts('N N1', Node)
    E, E1 = Consts('E E1', Epoch)

    def body(N, N1, E, E1):
        return And(
            Implies(
                And(
                    S.transfer(E, N),
                    Not(M.le(E, S.ep(N))),
                    S.transfer(E1, N1),
                    Or(
                        Not(N == N1),
                        Not(E == E1)
                    ),
                ),
                Not(M.le(E, E1))
            ),
            Implies(
                And(S.held(N), S.transfer(E1, N1)),
                M.le(E1, S.ep(N))
            )
        )

    if nodes is not None:
        return all([model.eval(body(n,n1,e,e1)) for n, n1, e, e1 in itertools.product(nodes, nodes, epochs, epochs)])

    inv = ForAll([N, N1, E, E1], body(N, N1, E, E1))
    return inv

def inv_fn_3(M, S, model=None, nodes=None, epochs=None):
    """
    conjecture (( transfer(E, N) & ~le(E, ep(N))) -> ~le(E, ep(N1)))
            & (held(N) -> le(ep(N1), ep(N)))
    """

    N, N1 = Consts('N N1', Node)
    E = Const('E', Epoch)

    def body(N, N1, E):
        return And(
            Implies(
                And(
                    S.transfer(E, N),
                    Not(M.le(E, S.ep(N))),
                ),
                Not(M.le(E, S.ep(N1)))
            ),
            Implies(
                S.held(N),
                M.le(S.ep(N1), S.ep(N))
            )
        )
    
    if nodes is not None:
        return all([model.eval(body(n,n1,e)) for n, n1, e in itertools.product(nodes, nodes, epochs)])

    inv = ForAll([N, N1, E], body(N, N1, E))
    return inv

swiss_invars = [inv_fn_0, inv_fn_1, inv_fn_2, inv_fn_3]

### DistAI invariants

In [6]:
invariants = """
le(E1, E2) & E1 ~= E2 -> le(E1,ep(N1)) | ~le(E2,ep(N1))
le(E1, E2) & E1 ~= E2 -> locked(E1,N1) | ~transfer(E1,N1) | ~transfer(E2,N1)
le(E1, E2) & E1 ~= E2 -> locked(E1,N1) | ~transfer(E1,N1) | ~le(E2,ep(N1))
le(E1, E2) & E1 ~= E2 -> le(E1,ep(N1)) | ~locked(E2,N1)
le(E1, E2) & E1 ~= E2 -> locked(E1,N1) | ~transfer(E1,N1) | ~locked(E2,N1)
le(E1, E2) & E1 ~= E2 -> le(E1,ep(N1)) | ~transfer(E1,N1) | ~transfer(E2,N1)
le(E1, E2) & E1 ~= E2 & N1 ~= N2 -> ~le(E2,ep(N1)) | ~le(ep(N1),ep(N2)) | ~le(ep(N2),ep(N1))
le(E1, E2) & E1 ~= E2 & N1 ~= N2 -> le(E1,ep(N1)) | le(ep(N1),ep(N2)) | ~locked(E2,N2)
le(E1, E2) & E1 ~= E2 & N1 ~= N2 -> le(E1,ep(N1)) | ~transfer(E1,N1) | ~locked(E2,N2)
le(E1, E2) & E1 ~= E2 & N1 ~= N2 -> locked(E1,N1) | ~transfer(E1,N1) | ~le(E2,ep(N2))
le(E1, E2) & E1 ~= E2 & N1 ~= N2 -> le(E1,ep(N1)) | ~held(N2) | ~transfer(E2,N1)
le(E1, E2) & E1 ~= E2 & N1 ~= N2 -> le(E1,ep(N1)) | ~transfer(E1,N1) | ~le(E2,ep(N2))
le(E1, E2) & E1 ~= E2 & N1 ~= N2 -> le(E1,ep(N1)) | ~locked(E2,N2) | ~le(ep(N2),ep(N1))
le(E1, E2) & E1 ~= E2 & N1 ~= N2 -> locked(E1,N1) | ~transfer(E1,N1) | ~transfer(E2,N2)
le(E1, E2) & E1 ~= E2 & N1 ~= N2 -> le(E1,ep(N1)) | le(ep(N1),ep(N2)) | ~le(E2,ep(N2))
le(E1, E2) & E1 ~= E2 & N1 ~= N2 -> le(E1,ep(N1)) | ~transfer(E1,N1) | ~transfer(E2,N2)
le(E1, E2) & E1 ~= E2 & N1 ~= N2 -> le(E1,ep(N1)) | ~transfer(E2,N1) | ~le(E2,ep(N2))
le(E1, E2) & E1 ~= E2 & N1 ~= N2 -> le(E1,ep(N1)) | ~le(ep(N2),ep(N1)) | ~le(E2,ep(N2))
le(E1, E2) & E1 ~= E2 & N1 ~= N2 -> locked(E1,N1) | ~transfer(E1,N1) | ~locked(E2,N2)
locked(E1,N1) | ~transfer(E1,N1) | ~le(E1,ep(N1))
locked(E1,N1) | ~held(N1) | ~transfer(E1,N1)
le(E1,ep(N1)) | ~held(N1) | ~transfer(E1,N1)
transfer(E1,N1) | ~locked(E1,N1)
le(E1,ep(N1)) | ~locked(E1,N1)
N1 ~= N2 -> ~le(ep(N1),ep(N2)) | ~le(ep(N2),ep(N1)) | ~first=N1
N1 ~= N2 -> le(E1,ep(N1)) | le(ep(N1),ep(N2)) | ~locked(E1,N2)
N1 ~= N2 -> le(E1,ep(N1)) | le(ep(N1),ep(N2)) | ~le(E1,ep(N2))
N1 ~= N2 -> le(ep(N1),ep(N2)) | ~held(N2)
N1 ~= N2 -> ~held(N1) | ~le(ep(N1),ep(N2))
N1 ~= N2 -> locked(E1,N1) | ~held(N2) | ~transfer(E1,N1)
N1 ~= N2 -> le(ep(N1),ep(N2)) | le(ep(N2),ep(N1))
N1 ~= N2 -> le(E1,ep(N1)) | ~locked(E1,N2) | ~le(ep(N2),ep(N1))
N1 ~= N2 -> ~held(N1) | ~held(N2)
N1 ~= N2 -> locked(E1,N1) | ~transfer(E1,N1) | ~le(E1,ep(N2))
N1 ~= N2 -> ~locked(E1,N1) | ~locked(E1,N2)
N1 ~= N2 -> ~first=N1 | ~first=N2
N1 ~= N2 -> ~transfer(E1,N1) | ~transfer(E1,N2)
N1 ~= N2 -> le(E1,ep(N1)) | ~held(N1) | ~transfer(E1,N2)
N1 ~= N2 -> ~transfer(E1,N1) | ~locked(E1,N2)
N1 ~= N2 -> ~locked(E1,N1) | ~le(ep(N1),ep(N2)) | ~le(ep(N2),ep(N1))
N1 ~= N2 -> le(E1,ep(N1)) | ~held(N1) | ~locked(E1,N2)
N1 ~= N2 -> le(E1,ep(N1)) | ~le(ep(N2),ep(N1)) | ~le(E1,ep(N2))
N1 ~= N2 -> le(E1,ep(N1)) | ~held(N1) | ~le(E1,ep(N2))
N1 ~= N2 -> le(E1,ep(N1)) | ~held(N2) | ~transfer(E1,N1)
N1 ~= N2 -> le(E1,ep(N1)) | ~transfer(E1,N1) | ~le(E1,ep(N2))
""".replace("~=", "!=").strip().split("\n")

def parse(inv_str, vars):
    inv_str = inv_str.strip()

    if inv_str.count("->") > 1:
        raise Exception("Too many ->")

    if '->' in inv_str:
        left, right = inv_str.split('->')
        return f"Implies({parse(left, vars)},{parse(right, vars)})"

    if "&" in inv_str:
        parts = inv_str.split("&")
        return f"And([{','.join([parse(p, vars) for p in parts])}])"

    if "|" in inv_str:
        parts = inv_str.split("|")
        return f"Or([{','.join([parse(p, vars) for p in parts])}])"

    if "~" in inv_str:
        return f"Not({parse(inv_str[1:], vars)})"

    if "!=" in inv_str:
        left, right = inv_str.split("!=")
        return f"{parse(left, vars)} != {parse(right, vars)}"
    
    if "=" in inv_str:
        left, right = inv_str.split("=")
        return f"{parse(left, vars)} == {parse(right, vars)}"

    if inv_str.endswith(")"):
        first_brace_idx = inv_str.index("(")
        name = inv_str[:first_brace_idx]
        args = inv_str[first_brace_idx+1:-1].split(",")
        prefix = "M." if name == 'le' else "S."
        return f"{prefix}{name}({','.join([parse(a, vars) for a in args])})"

    if inv_str == "first":
        return "M.first"

    vars.add(inv_str)
    return inv_str

def get_inv_fn(fn_name, inv_str, only_code=False):
    vars = set()
    inv_fn_str = parse(inv_str, vars)

    nodes = [v for v in vars if v.startswith("N")]
    epochs = [v for v in vars if v.startswith("E")]

    code = []
    if len(nodes) == 1:
        code += [nodes[0] + " = Const('" + nodes[0] + "', Node)"]
    elif len(nodes) > 1:
        code += [", ".join(nodes) + " = Consts('" + " ".join(nodes) + "', Node)"]
    
    if len(epochs) == 1:
        code += [epochs[0] + " = Const('" + epochs[0] + "', Epoch)"]
    elif len(epochs) > 1:
        code += [", ".join(epochs) + " = Consts('" + " ".join(epochs) + "', Epoch)"]
    
    code += ["inv = ForAll([" + ", ".join(vars) + "], " + inv_fn_str + ")"]
    code += ["return inv"]

    code = f"def {fn_name}(M, S):\n\t" + "\n\t".join(code)
    if only_code:
        return code

    ldict = {}
    exec(code)
    exec(f"ldict['fn'] = {fn_name}")
    return ldict['fn']

distai_invars = [get_inv_fn("inv_fn_" + str(i), inv) for i, inv in enumerate(invariants)]

In [7]:
M = DistLockModel()
S = M.get_state('pre')

print(invariants[0])
distai_invars[0](M, S)

le(E1, E2) & E1 != E2 -> le(E1,ep(N1)) | ~le(E2,ep(N1))


ForAll([E1, E2, N1],
       Implies(And(le(E1, E2), E1 != E2),
               Or(le(E1, pre.ep(N1)),
                  Not(le(E2, pre.ep(N1))))))

### All invariants

In [8]:
# all_invars = swiss_invars
all_invars = distai_invars

## Conditions

### Safety Condition

In [9]:
# Inv: safety property locked(E, N1) & locked(E, N2) -> N1 = N2

def get_safety_inv(M: DistLockModel, S: DistLockState, model=None, nodes=None, epochs=None):
    e1 = Const('e1', Epoch)
    n1, n2 = Consts('n1 n2', Node)

    def body(e1, n1, n2):
        return Implies(
            And(
                S.locked(e1, n1),
                S.locked(e1, n2)
            ),
            n1 == n2
        )

    if model is not None:
        return ([model.eval(body(e1, n1, n2)) for e1, n1, n2 in itertools.product(epochs, nodes, nodes)])

    Inv = ForAll([e1, n1, n2], body(e1, n1, n2))

    return Inv

M = DistLockModel()
S = M.get_state('pre')
get_safety_inv(M, S) # only for testing

ForAll([e1, n1, n2],
       Implies(And(pre.locked(e1, n1), pre.locked(e1, n2)),
               n1 == n2))

### Actions

In [10]:
# Grant action

def get_grant_action(M: DistLockModel, S1: DistLockState, S2: DistLockState, onlyPreq=False):
    """
    action grant(n1:node, n2:node, e:epoch) = {
        # release the lock and send a transfer message
        require held(n1);
        require ~le(e, ep(n1));   # jump to some strictly higher epoch
        transfer(e, n2) := true;
        held(n1) := false;
    }
    """
    e, EE = Consts('e EE', Epoch)
    n1, n2, NN = Consts('n1 n2 NN', Node)

    prec = Exists([n1, n2, e],
        And(
            S1.held(n1),
            Not(M.le(e, S1.ep(n1)))
        )
    )
    if onlyPreq:
        return prec

    AcceptAction = ForAll([n1, n2, e],
        Implies(
            # precondition
            And(
                S1.held(n1),
                Not(M.le(e, S1.ep(n1)))
            ),
            # postcondition
            And(
                # for everything, use S1's values unless there's a change.
                ForAll([EE, NN],
                    S2.transfer(EE, NN) == If(And(EE == e, NN == n2), True, S1.transfer(EE, NN))),
                ForAll([NN],
                    S2.held(NN) == If(NN == n1, False, S1.held(NN))),
                ForAll([EE, NN],
                    S2.locked(EE, NN) == S1.locked(EE, NN)),
                ForAll([NN],
                    S2.ep(NN) == S1.ep(NN))
            )
        )
    )

    return AcceptAction

M = DistLockModel()
S1 = M.get_state('pre')
S2 = M.get_state('post')
get_grant_action(M, S1, S2) # only for testing

ForAll([n1, n2, e],
       Implies(And(pre.held(n1), Not(le(e, pre.ep(n1)))),
               And(ForAll([EE, NN],
                          post.transfer(EE, NN) ==
                          If(And(EE == e, NN == n2),
                             True,
                             pre.transfer(EE, NN))),
                   ForAll(NN,
                          post.held(NN) ==
                          If(NN == n1, False, pre.held(NN))),
                   ForAll([EE, NN],
                          post.locked(EE, NN) ==
                          pre.locked(EE, NN)),
                   ForAll(NN, post.ep(NN) == pre.ep(NN)))))

In [11]:
# Accept action

def get_accept_action(M: DistLockModel, S1: DistLockState, S2: DistLockState, onlyPreq=False):
    """
    action accept(n:node, e:epoch) = {
        # receive a transfer message and take the lock, sending a locked message
        require transfer(e,n);
        if ~le(e, ep(n)) {
            held(n) := true;
            ep(n) := e;
            locked(e, n) := true;
        };
    }
    """
    e, EE = Consts('e EE', Epoch)
    n, NN = Consts('n NN', Node)

    prec = Exists([n, e],
        And(
            S1.transfer(e, n),
            Not(M.le(e, S1.ep(n)))
        )
    )
    if onlyPreq:
        return prec

    AcceptAction = ForAll([n, e],
        Implies(
            And(
                S1.transfer(e, n),
                Not(M.le(e, S1.ep(n)))
            ),
            And(
                ## Precise formulation
                ForAll([NN],
                    S2.held(NN) == If(n == NN, True, S1.held(NN))),
                ForAll([NN],
                    S2.ep(NN) == If(n == NN, e, S1.ep(NN))),
                ForAll([EE, NN],
                    S2.locked(EE, NN) == If(And(EE == e, NN == n), True, S1.locked(EE, NN))),
                ForAll([EE, NN],
                    S2.transfer(EE, NN) == S1.transfer(EE, NN)),
            )
        )
    )

    return AcceptAction

M = DistLockModel()
S1 = M.get_state('pre')
S2 = M.get_state('post')
get_accept_action(M, S1, S2) # only for testing

ForAll([n, e],
       Implies(And(pre.transfer(e, n),
                   Not(le(e, pre.ep(n)))),
               And(ForAll(NN,
                          post.held(NN) ==
                          If(n == NN, True, pre.held(NN))),
                   ForAll(NN,
                          post.ep(NN) ==
                          If(n == NN, e, pre.ep(NN))),
                   ForAll([EE, NN],
                          post.locked(EE, NN) ==
                          If(And(EE == e, NN == n),
                             True,
                             pre.locked(EE, NN))),
                   ForAll([EE, NN],
                          post.transfer(EE, NN) ==
                          pre.transfer(EE, NN)))))

In [12]:
all_invars = distai_invars[1:2]

## Verification

### Invariants(Init) -- Initial condition

In [13]:
M = DistLockModel()
S1 = M.get_state('pre')
S2 = M.get_state('post')

inv = lambda M, S: And(*[inv(M, S) for inv in all_invars[:]])

solver = Solver()
solver.assert_and_track(M.get_init_state_cond(), "1")
solver.assert_and_track(M.get_axioms(), "2")
solver.assert_and_track(Not(inv(M, M.get_state('init'))), "3")

assert solver.check() == unsat

^ unsat means that the initial state satisfies all invariants

### Invariants(Step) -- Inductiveness

In [85]:
def inductiveness_check():
    M = DistLockModel()
    S1 = M.get_state('pre')
    S2 = M.get_state('post')

    solver = Solver()
    solver.assert_and_track(M.get_init_state_cond(), "1")
    solver.assert_and_track(M.get_axioms(), "2")
    # solver.assert_and_track(inv(M, S1), "3")
    solver.assert_and_track(get_grant_action(M, S1, S2), "4")
    solver.assert_and_track(get_accept_action(M, S1, S2), "5")

    # important. This asserts that at least one of the actions is taken.
    # More specifically, it asserts that pre-condition for at least one of the actions is true.
    # Without this, z3 is free to come up with a non sensical Pre state. The reason is,
    # our transition relation is defined as prec -> post, and z3 is free to set prec == False.
    solver.assert_and_track(Or(get_grant_action(M, S1, S2, True), get_accept_action(M, S1, S2, True)), "6")

    # However, ideally, the above thing shouldn't be required and should be captured by the invariant.

    ## Asking z3 to solve it this way can cause it to hang.
    # solver.assert_and_track(Not(inv(M, S2)), "7")

    ## So we manually "unroll" as below.
    ## It is equivalent because AND(P, Not(q1, q2)) == Or(And(P, Not(q1)), And(P, Not(q2)))
    for i, inv_i in tqdm(enumerate(all_invars[:])):
        solver.push()
        for j, inv_j in enumerate(all_invars[:i+1]):
            solver.assert_and_track(inv_j(M, S1), "6.5_" + str(j))
        solver.assert_and_track(Not(inv_i(M, S2)), "7")
        if solver.check() == sat:
            print("Invariant set is not inductive")
            return solver.model()
        solver.pop()

    print("Invariant set is inductive.")
    return None

inductiveness_check()

0it [00:00, ?it/s]

Invariant set is not inductive


[one = Epoch!val!3,
 2 = True,
 7 = True,
 zero = Epoch!val!4,
 1 = True,
 5 = True,
 first = Node!val!0,
 4 = True,
 6 = True,
 6.5_0 = True,
 pre.locked = [else -> False],
 init.transfer = [else -> False],
 post.locked = [else -> False],
 le = [else ->
       Or(And(Var(0) == Epoch!val!4, Var(1) == Epoch!val!4),
          And(Not(Var(0) == Epoch!val!1),
              Not(Var(0) == Epoch!val!3),
              Not(Var(0) == Epoch!val!0),
              Not(Var(0) == Epoch!val!4),
              Var(1) == Epoch!val!0,
              Not(Var(1) == Epoch!val!4)),
          And(Not(Var(0) == Epoch!val!1),
              Not(Var(0) == Epoch!val!3),
              Not(Var(0) == Epoch!val!0),
              Not(Var(0) == Epoch!val!4),
              Not(Var(1) == Epoch!val!1),
              Not(Var(1) == Epoch!val!3),
              Not(Var(1) == Epoch!val!0),
              Not(Var(1) == Epoch!val!4)),
          And(Var(0) == Epoch!val!1,
              Not(Var(0) == Epoch!val!3),
              Not(Var(0) == Epoch!val!0),
              Not(Var(0) == Epoch!val!4),
              Var(1) == Epoch!val!0,
              Not(Var(1) == Epoch!val!4)),
          And(Var(0) == Epoch!val!0,
              Not(Var(0) == Epoch!val!4),
              Var(1) == Epoch!val!0,
              Not(Var(1) == Epoch!val!4)),
          And(Var(0) == Epoch!val!4,
              Var(1) == Epoch!val!3,
              Not(Var(1) == Epoch!val!0),
              Not(Var(1) == Epoch!val!4)),
          And(Var(0) == Epoch!val!4,
              Not(Var(1) == Epoch!val!1),
              Not(Var(1) == Epoch!val!3),
              Not(Var(1) == Epoch!val!0),
              Not(Var(1) == Epoch!val!4)),
          And(Var(0) == Epoch!val!3,
              Not(Var(0) == Epoch!val!0),
              Not(Var(0) == Epoch!val!4),
              Var(1) == Epoch!val!3,
              Not(Var(1) == Epoch!val!0),
              Not(Var(1) == Epoch!val!4)),
          And(Not(Var(0) == Epoch!val!1),
              Not(Var(0) == Epoch!val!3),
              Not(Var(0) == Epoch!val!0),
              Not(Var(0) == Epoch!val!4),
              Var(1) == Epoch!val!1,
              Not(Var(1) == Epoch!val!3),
              Not(Var(1) == Epoch!val!0),
              Not(Var(1) == Epoch!val!4)),
          And(Var(0) == Epoch!val!3,
              Not(Var(0) == Epoch!val!0),
              Not(Var(0) == Epoch!val!4),
              Not(Var(1) == Epoch!val!1),
              Not(Var(1) == Epoch!val!3),
              Not(Var(1) == Epoch!val!0),
              Not(Var(1) == Epoch!val!4)),
          And(Var(0) == Epoch!val!3,
              Not(Var(0) == Epoch!val!0),
              Not(Var(0) == Epoch!val!4),
              Var(1) == Epoch!val!0,
              Not(Var(1) == Epoch!val!4)),
          And(Var(0) == Epoch!val!4,
              Var(1) == Epoch!val!0,
              Not(Var(1) == Epoch!val!4)),
          And(Var(0) == Epoch!val!4,
              Var(1) == Epoch!val!1,
              Not(Var(1) == Epoch!val!3),
              Not(Var(1) == Epoch!val!0),
              Not(Var(1) == Epoch!val!4)),
          And(Var(0) == Epoch!val!3,
              Not(Var(0) == Epoch!val!0),
              Not(Var(0) == Epoch!val!4),
              Var(1) == Epoch!val!1,
              Not(Var(1) == Epoch!val!3),
              Not(Var(1) == Epoch!val!0),
              Not(Var(1) == Epoch!val!4)),
          And(Var(0) == Epoch!val!1,
              Not(Var(0) == Epoch!val!3),
              Not(Var(0) == Epoch!val!0),
              Not(Var(0) == Epoch!val!4),
              Var(1) == Epoch!val!1,
              Not(Var(1) == Epoch!val!3),
              Not(Var(1) == Epoch!val!0),
              Not(Var(1) == Epoch!val!4)))],
 init.ep = [else -> Epoch!val!3],
 post.held = [else -> False],
 pre.ep = [else -> Epoch!val!1],
 init.locked = [else -> False],
 init.held = [else -> True],
 post.ep = [else -> Epoch!val!1],
 pre.transfer = [else ->
                 And(Not(Var(0) == Epoch!val!1),
                     Not(Var(0) == Epoch!val!3),
                   

In [93]:
def get_formulas(model, S):
    """Converts relations of a state S from z3 function to a ground formula in the model's universe """

    node_univ, epoch_univ = model.get_universe(Node), model.get_universe(Epoch)
    univ = {Node: node_univ, Epoch: epoch_univ}
    decls = model.decls()

    #global_decls = must be included in general
    decls_ = [d for d in decls if d.name().startswith(S.name) or "." not in d.name()]
    formulas = {}
    for d in decls_:
        universes = [univ[d.domain(i)] for i in range(d.arity())]
        all_args = itertools.product(*universes)
        # TODO Optimization: remove those for which model_completion was necessary (i.e, don't care inputs)
        func_as_formula = And([d(*args) == model.eval(d(*args), model_completion=True) for args in all_args])
        formulas[d.name()] = func_as_formula

    return formulas, And(*formulas.values())

m = inductiveness_check()
if m is not None:
    pre_formulas_dict, pre_formula = get_formulas(m, S1)
    post_formulas_dict, post_formula = get_formulas(m, S2)

0it [00:00, ?it/s]

Invariant set is not inductive


In [94]:
print(pre_formula.sexpr())

(and (= zero Epoch!val!4)
     (= |4| true)
     (= |1| true)
     (= |2| true)
     (= one Epoch!val!3)
     (= |6| true)
     (= |7| true)
     (= first Node!val!0)
     (= |5| true)
     (= (pre.locked Epoch!val!3 Node!val!0) false)
     (= (pre.locked Epoch!val!4 Node!val!0) false)
     (= (pre.locked Epoch!val!2 Node!val!0) false)
     (= (pre.locked Epoch!val!5 Node!val!0) false)
     (= (pre.locked Epoch!val!1 Node!val!0) false)
     (= (pre.locked Epoch!val!0 Node!val!0) false)
     (= (le Epoch!val!3 Epoch!val!3) true)
     (= (le Epoch!val!3 Epoch!val!4) false)
     (= (le Epoch!val!3 Epoch!val!2) true)
     (= (le Epoch!val!3 Epoch!val!5) true)
     (= (le Epoch!val!3 Epoch!val!1) true)
     (= (le Epoch!val!3 Epoch!val!0) true)
     (= (le Epoch!val!4 Epoch!val!3) true)
     (= (le Epoch!val!4 Epoch!val!4) true)
     (= (le Epoch!val!4 Epoch!val!2) true)
     (= (le Epoch!val!4 Epoch!val!5) true)
     (= (le Epoch!val!4 Epoch!val!1) true)
     (= (le Epoch!val!4 Epoch!val!0

In [87]:
print(m.sexpr())

;; universe for Epoch:
;;   Epoch!val!3 Epoch!val!4 Epoch!val!2 Epoch!val!5 Epoch!val!1 Epoch!val!0 
;; -----------
;; definitions for universe elements:
(declare-fun Epoch!val!3 () Epoch)
(declare-fun Epoch!val!4 () Epoch)
(declare-fun Epoch!val!2 () Epoch)
(declare-fun Epoch!val!5 () Epoch)
(declare-fun Epoch!val!1 () Epoch)
(declare-fun Epoch!val!0 () Epoch)
;; cardinality constraint:
(forall ((x Epoch))
        (or (= x Epoch!val!3)
            (= x Epoch!val!4)
            (= x Epoch!val!2)
            (= x Epoch!val!5)
            (= x Epoch!val!1)
            (= x Epoch!val!0)))
;; -----------
;; universe for Node:
;;   Node!val!0 
;; -----------
;; definitions for universe elements:
(declare-fun Node!val!0 () Node)
;; cardinality constraint:
(forall ((x Node)) (= x Node!val!0))
;; -----------
(define-fun one () Epoch
  Epoch!val!5)
(define-fun |4| () Bool
  true)
(define-fun |2| () Bool
  true)
(define-fun zero () Epoch
  Epoch!val!4)
(define-fun |7| () Bool
  true)
(define-fun

Todo:

(0) Programmatically generate the synth condition
   - Sorts
   - Universe
   - Relations
   - synth-fun (Grammar)
   - Constraints
      
(1) Template
- iterate over different signatures of invariant
- for each signature, iterate over different quantifiers<br>
   (or)
- iterate over different "templates"

(2) Write the CEGIS loop
   - quantify the synthesized invariant
   - verify
   - synthesize again

In [64]:
def synth_invar(quantifiers, sorts, model):
    """
    Example:
    quantifiers = ['EXISTS', 'FORALL']
    sorts       = [Node, Epoch]
    universes   = [node_univ, epoch_univ]

    invar = Function('invar', Node, Epoch, BoolSort())
    ForAll(N), Exists(E) such that invar(N, E) == True
    And([
        Or([
            invar(n, e)
            for e in epoch_univ
        ])
        for n in node_univ
    ])
    """
    synthesized_inv = Function('inv', *(sorts + [BoolSort()]))

    universes = [model.get_universe(s) for s in sorts]

    def helper(qList, uList, args):
        q, u = qList[0], uList[0]
        assert q in ['FORALL', 'EXISTS']

        Q = And if q.upper() == 'FORALL' else Or

        if len(qList) == 1 and len(uList) == 1:
            # print(q, u, Q([synthesized_inv(*(args + [elem])) for elem in u]))
            return Q([synthesized_inv(*(args + [elem])) for elem in u])
        
        # print(q, u, Q([helper(qList[1:], uList[1:], args + [elem]) for elem in u]))
        return Q([helper(qList[1:], uList[1:], args + [elem]) for elem in u])

    # return synthesized_inv, helper(quantifiers, universes, [])
    return helper(quantifiers, universes, [])

# inv_expr = synth_invar(['FORALL', 'FORALL'], [Epoch, Epoch], m)
inv_expr = synth_invar(['FORALL', 'EXISTS'], [Epoch, Epoch], m)
print(inv_expr.sexpr())

(and (or (inv Epoch!val!3 Epoch!val!3)
         (inv Epoch!val!3 Epoch!val!2)
         (inv Epoch!val!3 Epoch!val!1)
         (inv Epoch!val!3 Epoch!val!0))
     (or (inv Epoch!val!2 Epoch!val!3)
         (inv Epoch!val!2 Epoch!val!2)
         (inv Epoch!val!2 Epoch!val!1)
         (inv Epoch!val!2 Epoch!val!0))
     (or (inv Epoch!val!1 Epoch!val!3)
         (inv Epoch!val!1 Epoch!val!2)
         (inv Epoch!val!1 Epoch!val!1)
         (inv Epoch!val!1 Epoch!val!0))
     (or (inv Epoch!val!0 Epoch!val!3)
         (inv Epoch!val!0 Epoch!val!2)
         (inv Epoch!val!0 Epoch!val!1)
         (inv Epoch!val!0 Epoch!val!0)))


In [65]:
# synthesized_inv = Function('inv', Node, BoolSort())

# inv_expr = synth_invar(['FORALL', 'EXISTS'], [Epoch, Epoch], m)
inv_expr = synth_invar(['FORALL', 'EXISTS'], [Node, Epoch], m)

# \forall n
ice_constraint_1 = Implies(pre_formula, Not(inv_expr))
ice_constraint_1_sexpr = (ice_constraint_1.sexpr().replace('pre.',''))
print(ice_constraint_1_sexpr)
# ice_constraint_2 = And(pre_formula, post_formula)

# ice_constraint = Or(ice_constraint_1, ice_constraint_2)
# ice_constraint

(let ((a!1 (not (and (or (inv Node!val!0 Epoch!val!3)
                         (inv Node!val!0 Epoch!val!2)
                         (inv Node!val!0 Epoch!val!1)
                         (inv Node!val!0 Epoch!val!0))))))
  (=> (and (= (ep Node!val!0) Epoch!val!3)
           (= (locked Epoch!val!3 Node!val!0) false)
           (= (locked Epoch!val!2 Node!val!0) false)
           (= (locked Epoch!val!1 Node!val!0) false)
           (= (locked Epoch!val!0 Node!val!0) false)
           (= (held Node!val!0) true)
           (= (transfer Epoch!val!3 Node!val!0) false)
           (= (transfer Epoch!val!2 Node!val!0) false)
           (= (transfer Epoch!val!1 Node!val!0) false)
           (= (transfer Epoch!val!0 Node!val!0) true))
      a!1))


In [66]:
synth_str = f"""
(set-logic ALL)

(declare-sort Node)
(declare-sort Epoch)

;; universe for Epoch:
;;   Epoch!val!3 Epoch!val!4 Epoch!val!2 Epoch!val!1 Epoch!val!0 
;; -----------
;; definitions for universe elements:
(declare-fun Epoch!val!3 () Epoch)
(declare-fun Epoch!val!4 () Epoch)
(declare-fun Epoch!val!2 () Epoch)
(declare-fun Epoch!val!1 () Epoch)
(declare-fun Epoch!val!0 () Epoch)
;; cardinality constraint: - not having here

;; -----------
;; universe for Node:
;;   Node!val!0 
;; -----------
;; definitions for universe elements:
(declare-fun Node!val!0 () Node)
;; cardinality constraint: - not having here

(declare-fun held (Node) Bool)
(declare-fun locked (Epoch Node) Bool)
(declare-fun transfer (Epoch Node) Bool)
(declare-fun ep (Node) Epoch)
(declare-fun le (Epoch Epoch) Bool)

;;(synth-fun inv ((e1 Epoch) (e2 Epoch)) Bool
(synth-fun inv ((n1 Node) (e1 Epoch)) Bool
    
    ;; Declare the non-terminals that would be used in the grammar
    ((Start Bool) (Atom Bool) (Node_ Node) (Epoch_ Epoch))

    ;; Define the grammar for allowed implementations of max2
    (
        (Start Bool 
               (
                (and Atom Atom)
                (or Atom Atom)
                (=> Atom Atom)
                (not Atom)
               )
        )

        (Atom Bool 
               (
                (held Node_)
                (locked Epoch_ Node_)
                (transfer Epoch_ Node_)
                (le Epoch_ Epoch_)
               )
        )

        (Node_ Node
            (
                n1
            )
        )

        (Epoch_ Epoch
            (
                e1
                ;; e2
                (ep Node_)
            )
        )
     
    )
)

(constraint {ice_constraint_1_sexpr})

(check-synth)
"""

# write synth_str to file
with open('synth.smt2', 'w') as f:
    f.write(synth_str)

# run minisy (with the above file)
# spawn childprocess or subprocess and read the stdout


print(synth_str)
with open('test_synth.sy','w') as synth_file:
    synth_file.write(synth_str)


(set-logic ALL)

(declare-sort Node)
(declare-sort Epoch)

;; universe for Epoch:
;;   Epoch!val!3 Epoch!val!4 Epoch!val!2 Epoch!val!1 Epoch!val!0 
;; -----------
;; definitions for universe elements:
(declare-fun Epoch!val!3 () Epoch)
(declare-fun Epoch!val!4 () Epoch)
(declare-fun Epoch!val!2 () Epoch)
(declare-fun Epoch!val!1 () Epoch)
(declare-fun Epoch!val!0 () Epoch)
;; cardinality constraint: - not having here

;; -----------
;; universe for Node:
;;   Node!val!0 
;; -----------
;; definitions for universe elements:
(declare-fun Node!val!0 () Node)
;; cardinality constraint: - not having here

(declare-fun held (Node) Bool)
(declare-fun locked (Epoch Node) Bool)
(declare-fun transfer (Epoch Node) Bool)
(declare-fun ep (Node) Epoch)
(declare-fun le (Epoch Epoch) Bool)

;;(synth-fun inv ((e1 Epoch) (e2 Epoch)) Bool
(synth-fun inv ((n1 Node) (e1 Epoch)) Bool
    
    ;; Declare the non-terminals that would be used in the grammar
    ((Start Bool) (Atom Bool) (Node_ Node) (Epoch_ E

In [59]:
def parse_inv_defn(quantifiers, sorts, defn):
    defn += "\n"
    sort_counts = {}

    indent = 1
    q_opens = ""
    q_closes = ""
    args = []
    for q, s in zip(quantifiers, sorts):
        sort_counts[s] = sort_counts.get(s, 0) + 1
        count = sort_counts[s]
        pref = s.name()[0].lower()
        name = pref + str(count)
        defn += f"(declare-fun {name} () {s.name()})\n"
        args += [name]
        q_opens += f"{('   ')*indent}({q.lower()} (({name} {s.name()}))\n"
        q_closes = f"{('   ')*indent})\n" + q_closes
        indent += 1
    
    defn += f"(assert\n{q_opens}{('   ')*indent}(inv {' '.join(args)})\n{q_closes})\n"
    # print(defn)
    # return

    def inv(M, S):
        sorts = {'Node': Node, 'Epoch': Epoch}
        decls = {'held': S.held, 'locked': S.locked, 'transfer': S.transfer,
                 'ep': S.ep, 'le': M.le}
        return parse_smt2_string(defn, sorts=sorts, decls=decls)[0]
    
    return inv

defn = '(define-fun inv ((n1 Node) (e1 Epoch)) Bool\n(or (le e1 (ep n1)) (le (ep n1) (ep n1)))\n)'
inv_fn = parse_inv_defn(['FORALL', 'FORALL'], [Node, Epoch], defn)
# f = parse_smt2_string('(define-fun inv ((n1 Node) (e1 Epoch)) Bool\n(or (le e1 (ep n1)) (le (ep n1) (ep n1)))\n)\n (declare-fun n1 () Node) (declare-fun e1 () Epoch) (assert (inv n1 e1))')
# type(f)

In [60]:
inv_fn(M, S1)

ForAll(n1,
       ForAll(e1,
              Or(le(e1, pre.ep(n1)),
                 le(pre.ep(n1), pre.ep(n1)))))

In [69]:
import subprocess

def run_minisy(synth_file):
    out = subprocess.check_output(f'source ~/.zshrc; minisy {synth_file} --num-solutions=2', shell=True, executable="/bin/zsh", encoding='utf-8')
    # print(out)
    lines = out.split('\n')
    defs = []
    for line in lines:
        if line == 'sat':
            continue
        if line == 'unsat':
            break
        if line.startswith('(define-fun'):
            defs.append(line)
        else:
            # print(line, defs)
            defs[-1] += '\n' + line
    return [d.strip() for d in defs]

run_minisy('test_synth.sy')

['(define-fun inv ((n1 Node) (e1 Epoch)) Bool\n(=> (held n1) (le e1 e1))\n)',
 '(define-fun inv ((n1 Node) (e1 Epoch)) Bool\n(and (le e1 (ep n1)) (le (ep n1) (ep n1)))\n)']

In [ ]:
def cegis_loop(iters):
    while iters > 0:
        iters -= 1
        print(f'iter {iters}')
        run_minisy('test_synth.sy')
        inv_fn(M, S1)


['(define-fun inv ((n1 Node) (e1 Epoch)) Bool\n(or (le e1 (ep n1)) (le (ep n1) (ep n1)))\n)',
 '(define-fun inv ((n1 Node) (e1 Epoch)) Bool\n(or (le e1 (ep n1)) (held n1))\n)']

### Invariants => Safety

The stuff below gives unsat (as expected) for DistAI invariants, but not for Swiss invariants. Not sure why. But essentially, invariants allow a state like this one:

```
(define-fun pre.locked ((x!0 Epoch) (x!1 Node)) Bool
  (or (and (not (= x!0 Epoch!val!3))
           (not (= x!0 Epoch!val!1))
           (not (= x!0 Epoch!val!4))
           (not (= x!0 Epoch!val!2))
           (= x!1 Node!val!1))
      (and (not (= x!0 Epoch!val!3))
           (not (= x!0 Epoch!val!1))
           (not (= x!0 Epoch!val!4))
           (not (= x!0 Epoch!val!2))
           (not (= x!1 Node!val!1)))))
```

Which is essentially `True` if `(x!0 == Epoch!val!0)` regardless of the node's value. This shouldn't be allowed!

In [ ]:
M = DistLockModel()
S = M.get_state('pre')

inv = lambda M, S: And(*[inv(M, S) for inv in all_invars[:]])

solver = Solver()
solver.assert_and_track(M.get_init_state_cond(), "1")
solver.assert_and_track(M.get_axioms(), "2")
solver.assert_and_track(inv(M, S), "3")

## Safety VC:
solver.assert_and_track(Not(get_safety_inv(M, S)), "8")

assert solver.check() == unsat

In [ ]:
model = solver.model()
d_model = M.get_interp(model)
d_model

In [ ]:
all_nodes, all_epochs = model.get_universe(Node), model.get_universe(Epoch)

In [ ]:
all([all_invars[i](M, S, model, all_nodes, all_epochs) for i in range(len(all_invars))])

In [ ]:
all_nodes, all_epochs

In [ ]:
get_safety_inv(M, S)

In [ ]:
list(itertools.product(all_epochs, all_nodes, all_nodes))

In [ ]:
get_safety_inv(M, S, model, all_nodes, all_epochs)

In [ ]:
def cmp(e1, e2):
    return model.eval(M.le(e1, e2))

# bubble sort
def bubble_sort(l):
    for i in range(len(l)):
        for j in range(len(l)-1):
            if cmp(l[j], l[j+1]) == False:
                l[j], l[j+1] = l[j+1], l[j]
    return l

sorted_epochs = bubble_sort(all_epochs)
sorted_epochs

In [ ]:
print(solver.model().sexpr())